In [1]:
include("BBVI_modeling.jl")

In [2]:
using RCall

R"""
load("data.RData")
"""
data = @rget data
Y = data[:Y]
Q = convert(Matrix{Int64}, data[:Q])
obs = DCMObs(Y, Q);

In [4]:
a0 = 1e-2
b0 = 1e-4
d0 = ones(size(obs.D[1], 1))
M = 100
model = DCModel(obs, d0, a0, b0, M);

In [22]:
using BenchmarkTools

res = Vector{Vector}(undef, 30)
D = obs.D
mu = model.mu_star

for k in 1:30
    res[k] = Vector{Float64}(undef, 4)
end

@benchmark $res .= $D .* $mu

BenchmarkTools.Trial: 10000 samples with 61 evaluations.
 Range (min … max):  837.705 ns …  2.084 ms  ┊ GC (min … max):  0.00% … 99.91%
 Time  (median):     991.803 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.512 μs ± 20.865 μs  ┊ GC (mean ± σ):  17.20% ±  4.34%

   ▅█▇▄▆▂                          ▁▁                           
  ▄██████▆▄▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▃▄▆▇██▇▅▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  838 ns          Histogram: frequency by time         2.44 μs <

 Memory estimate: 2.81 KiB, allocs estimate: 30.

In [23]:
function fill_res!(res, D, mu)
    for k in 1:30
        mul!(res[k], D[k], mu[k])
    end
end

@benchmark fill_res!($res, $D, $mu)

BenchmarkTools.Trial: 10000 samples with 150 evaluations.
 Range (min … max):  684.667 ns …  1.501 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     756.667 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   768.109 ns ± 41.731 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                  █▄    ▂                                       
  ▂▁▂▂▂▂▂▂▂▃▃▃▆█▅▃██▆▇▆▅█▄▄▅▅▃▄▃▃▃▂▂▂▃▃▂▂▂▃▃▂▂▂▂▂▂▂▃▃▂▂▃▂▂▂▂▃▂ ▃
  685 ns          Histogram: frequency by time          906 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.